# An extensive walk through `DownClim`

```{note}
Although all the described methods and functions describe below are fully functional, this is not the recommended way to use the library, as you don't fully take advantage of the workflow and abstractions provided. Take a look at other notebooks from the `examples` section to see how to use `DownClim` in a more efficient way.

However, maybe at some point you'll find yourself your own way to use `DownClim` !
```

## Definition of the workflow

In this intentiionaly simplified example, we will use the `DownClim` library to:
- download `CHELSA` data for the baseline period 1980-1981
- download `CHIRPS` and `GSHTD` data for the evaluation period 2005-2006
- download `CORDEX` and `CMIP6` data for the baseline, evaluation and projection periods

## Authentication

First, we need to authenticate to Earth Engine to retrieve data from `GSHTD`, `CHIRPS` and `CMIP6`.

Although we also need to authenticate to `ESGF` for `CORDEX` data, login information can be provided in a separate file.

In [ ]:
import ee

ee.Authenticate()
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com", project="downclim")


Successfully saved authorization token.


## Areas of interest

We first need to define the areas of interest. This will define the boundaries for downloading the data and the area for which we will be predicting the downscaling.

There are multiple ways to define the areas of interest (cf. api link to `get_aoi`). 

In [4]:
from __future__ import annotations
from downclim.aoi import get_aoi, get_aoi_informations

aoi1 = get_aoi("Vanuatu")
aoi2 = get_aoi((10, 10, 20, 20, "box"))

## Download CHELSA data

In [3]:
from downclim.dataset.chelsa2 import get_chelsa2

get_chelsa2(
    aoi=[aoi1, aoi2],
    variable=["pr", "tas", "tasmin", "tasmax"],
    period=(1980, 1981),
    keep_tmp_dir=True,
)

Getting year "1980" for variables "pr" and areas of interest : "['Vanuatu', 'box']"
Getting year "1981" for variables "pr" and areas of interest : "['Vanuatu', 'box']"
Concatenating data for area of interest : Vanuatu
Concatenating data for area of interest : Vanuatu
saving file ./results/tmp/chelsa/chelsa_Vanuatu_pr_1981.nc
Concatenating data for area of interest : box
saving file ./results/tmp/chelsa/chelsa_Vanuatu_pr_1980.nc
Concatenating data for area of interest : box
saving file ./results/tmp/chelsa/chelsa_box_pr_1981.nc
saving file ./results/tmp/chelsa/chelsa_box_pr_1980.nc
Getting year "1980" for variables "tas" and areas of interest : "['Vanuatu', 'box']"Getting year "1981" for variables "tas" and areas of interest : "['Vanuatu', 'box']"

Concatenating data for area of interest : Vanuatu
Concatenating data for area of interest : Vanuatu
saving file ./results/tmp/chelsa/chelsa_Vanuatu_tas_1981.nc
Concatenating data for area of interest : box
saving file ./results/tmp/chelsa/che

## Downalod CHIRPS and GSHTD data

In [9]:
from downclim.dataset.chirps import get_chirps
from downclim.dataset.gshtd import get_gshtd

get_chirps(
    aoi=[aoi1, aoi2],
    period=(2006, 2007),
    project = "downclim", # project name for Earth Engine
)

get_gshtd(
    aoi=[aoi1, aoi2],
    variable=["tas", "tasmin", "tasmax"],
    period=(2006, 2007),
)


Already connected to Earth Engine with project 'downclim'.
Getting CHIRPS data for period : "(2006, 2007)" and area of interest : "Vanuatu"
Getting CHIRPS data for period : "(2006, 2007)" and area of interest : "box"
Already connected to Earth Engine with project 'downclim'.
Getting GSHTD data for period : "(2006, 2007)" and variable : "tas" on area of interest : "Vanuatu"
Getting GSHTD data for period : "(2006, 2007)" and variable : "tasmin" on area of interest : "Vanuatu"
Getting GSHTD data for period : "(2006, 2007)" and variable : "tasmax" on area of interest : "Vanuatu"
Getting GSHTD data for period : "(2006, 2007)" and variable : "tas" on area of interest : "box"


Getting GSHTD data for period : "(2006, 2007)" and variable : "tasmin" on area of interest : "box"


Getting GSHTD data for period : "(2006, 2007)" and variable : "tasmax" on area of interest : "box"


## Download climate simulations

### Download `CORDEX` data

In [ ]:
from downclim.dataset.cordex import (
    CORDEXContext,
    get_cordex_from_list,
    get_download_scripts,
    list_available_cordex_simulations,
)

# Define the research context for CORDEX data
cordex_context = CORDEXContext(
    domain=["AUS-22", "AFR-44"],
    experiment=["historical", "rcp26", "rcp85"],
    frequency="mon",
    variable=["pr", "tas"],
)

# Use the previously defined context to list available simulations
# ! This step requires ESGF credentials
cordex_simulations = list_available_cordex_simulations(
    cordex_context, esgf_credential="../../config/esgf_credential.yaml"
)

# Retrieve download scripts for the available simulations
cordex_simulations = get_download_scripts(cordex_simulations, esgf_credential="config/esgf_credential.yaml")

# Save the list of simulations to a CSV file. This can be useful if you want to perform hand-selection.
cordex_simulations.to_csv("results/cordex/cordex_simulations.csv")

get_cordex_from_list(
    aoi=[aoi1, aoi2],
    cordex_simulations=cordex_simulations,
    historical_period=(1980, 1981),
    evaluation_period=(2006, 2007),
    projection_period=(2071, 2072),
    output_dir="./results/cordex",
    tmp_dir = "./results/tmp/cordex",
    keep_tmp_dir = True,
    esgf_credential="config/esgf_credential.yaml"
)


FileNotFoundError: [Errno 2] No such file or directory: 'config/esgf_credential.yaml'

### Download `CMIP6` data

In [ ]:
from downclim.dataset.cmip6 import (
                                    CMIP6Context,
                                    get_cmip6,
                                    list_available_cmip6_simulations,
)

cmip6_context = CMIP6Context(
    project=["ScenarioMIP", "CMIP"],
    institute=["NOAA-GFDL", "CMCC"],
    experiment=["ssp126", "historical"],
    ensemble="r1i1p1f1",
    frequency="mon",
    variable=["tas", "pr"],
    grid_label="gn",
)
cmip6_simulations = list_available_cmip6_simulations(cmip6_context)
cmip6_simulations.to_csv("results/cmip6/cmip6_simulations.csv")

get_cmip6_from_list(
    aoi=[aoi1, aoi2],
    cmip6_simulations=cmip6_simulations,
    historical_period=(1980, 1981),
    evaluation_period=(2006, 2007),
    projection_period=(2071, 2072),
    output_dir="./results/cmip6",
)

OSError: Cannot save file into a non-existent directory: 'results/cmip6'

## Evaluation

## Downscaling